In [2]:
import pandas as pd

## Paso 1: Cargar y explorar el dataset

In [ ]:
# Cargar el dataset
data = pd.read_csv('stroke_data.csv')

# Explorar los primeros registros
print(data.head())

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


## Paso 2: Preprocesamiento de datos
- Imputación de valores faltantes

- Codificación de variables categóricas

- Escalado de características

In [24]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificar características
numerical_features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']
categorical_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Crear transformadores
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Aplicar transformadores a las columnas correspondientes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])


## Paso 3: División de datos en conjuntos de entrenamiento, validación y prueba

In [25]:
from sklearn.model_selection import train_test_split

# Definir características (X) y la variable objetivo (Y)
X = data.drop('stroke', axis=1)
Y = data['stroke']

# Dividir los datos en conjunto de entrenamiento y conjunto temporal (test + val)
X_train_temp, X_test, Y_train_temp, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# Dividir el conjunto temporal en validación y prueba
X_train, X_val, Y_train, Y_val = train_test_split(X_train_temp, Y_train_temp, test_size=0.25, stratify=Y_train_temp, random_state=42)

print(Y_train.sum()/Y_train.shape[0])

0.048597521200260924


## Paso 4: Entrenamiento del modelo
- Vamos a usar un modelo de clasificación como un Random Forest, cuidando el filtrado de información.

In [26]:
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, balanced_accuracy_score


# Crear pipeline completo con preprocesador y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))])

# Entrenar el modelo
model.fit(X_train, Y_train)

# Evaluar en el conjunto de validación
Y_pred_val = model.predict(X_val)
print(classification_report(Y_val, Y_pred_val))


# Calcular balanced accuracy
balanced_accuracy = balanced_accuracy_score(Y_val, Y_pred_val)
print(f"Balanced Accuracy: {balanced_accuracy:.2f}")
print()
print("*** classification_report_imbalanced ***")
# Evaluar en el conjunto de validación
Y_pred_val = model.predict(X_val)
print(classification_report_imbalanced(Y_val, Y_pred_val))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97       972
           1       0.33      0.02      0.04        50

    accuracy                           0.95      1022
   macro avg       0.64      0.51      0.51      1022
weighted avg       0.92      0.95      0.93      1022

Balanced Accuracy: 0.51

*** classification_report_imbalanced ***
                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      1.00      0.02      0.97      0.14      0.02       972
          1       0.33      0.02      1.00      0.04      0.14      0.02        50

avg / total       0.92      0.95      0.07      0.93      0.14      0.02      1022



## Paso 5: Evaluación final
- Evaluar el modelo en el conjunto de prueba para obtener una estimación más realista de su rendimiento.

In [27]:
# Evaluar en el conjunto de prueba
Y_pred_test = model.predict(X_test)
print(classification_report(Y_test, Y_pred_test))

# Calcular balanced accuracy
balanced_accuracy_test = balanced_accuracy_score(Y_test, Y_pred_test)
print(f"Balanced Accuracy (Test Set): {balanced_accuracy_test:.2f}")

# Evaluar en el conjunto de prueba
Y_pred_test = model.predict(X_test)
print(classification_report_imbalanced(Y_test, Y_pred_test))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       972
           1       0.00      0.00      0.00        50

    accuracy                           0.95      1022
   macro avg       0.48      0.50      0.49      1022
weighted avg       0.90      0.95      0.93      1022

Balanced Accuracy (Test Set): 0.50
                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      1.00      0.00      0.97      0.00      0.00       972
          1       0.00      0.00      1.00      0.00      0.00      0.00        50

avg / total       0.90      0.95      0.05      0.93      0.00      0.00      1022



Paso 1: Implementar Sobremuestreo (Oversampling) con SMOTE
SMOTE (Synthetic Minority Over-sampling Technique) genera ejemplos sintéticos para la clase minoritaria.

In [28]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

print("Distribución después de SMOTE:")
print(Y_train_resampled.value_counts())


ValueError: could not convert string to float: 'Male'

Paso 2: Implementar Submuestreo (Undersampling) con RandomUnderSampler
RandomUnderSampler reduce la cantidad de ejemplos de la clase mayoritaria.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Aplicar RandomUnderSampler al conjunto de entrenamiento
rus = RandomUnderSampler(random_state=42)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)

print("Distribución después de RandomUnderSampler:")
print(Y_train_resampled.value_counts())


Paso 3: Entrenamiento del modelo con los datos resampleados
Vamos a entrenar el modelo con los datos equilibrados generados por SMOTE y RandomUnderSampler.

In [ ]:
# Entrenar el modelo con SMOTE
model_smote = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))])

model_smote.fit(X_train_resampled, Y_train_resampled)

# Evaluar en el conjunto de validación
Y_pred_val_smote = model_smote.predict(X_val)
print("Informe de clasificación con SMOTE:")
print(classification_report_imbalanced(Y_val, Y_pred_val_smote))

# Entrenar el modelo con RandomUnderSampler
model_rus = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))])

model_rus.fit(X_train_resampled, Y_train_resampled)

# Evaluar en el conjunto de validación
Y_pred_val_rus = model_rus.predict(X_val)
print("Informe de clasificación con RandomUnderSampler:")
print(classification_report_imbalanced(Y_val, Y_pred_val_rus))


Paso 4: Evaluación final
Evaluar ambos modelos en el conjunto de prueba para comparar su rendimiento.

In [ ]:
# Evaluar el modelo con SMOTE en el conjunto de prueba
Y_pred_test_smote = model_smote.predict(X_test)
print("Informe de clasificación con SMOTE en el conjunto de prueba:")
print(classification_report_imbalanced(Y_test, Y_pred_test_smote))

# Evaluar el modelo con RandomUnderSampler en el conjunto de prueba
Y_pred_test_rus = model_rus.predict(X_test)
print("Informe de clasificación con RandomUnderSampler en el conjunto de prueba:")
print(classification_report_imbalanced(Y_test, Y_pred_test_rus))
